# Notebook `Data Extractor`
***

In [1]:
from pathlib import Path
from src.modules.scraping_functions import *
import polars as pl

**Note** : dans le folder `data`, tu trouveras le fichier `wine_links.csv`, il contient l'ensemble des href trouvés après le lancement du module `scraper.py`

- Dans le fichier actuel, il y en a *540*, mais en modifiant les **range** de la fonction `create_urls_browse_list` du module `scraping_functions.py`, tu peux récupérer l'ensemble des hrefs.
- `⚠` ! Pour récupérer les hrefs il faut obligatoirement faire tourner le module `scraper.py` car les hrefs sont générés post-javascript (évidemment ça ne marche pas dans les notebooks...) mais pour les caractéristiques des vins, prix et autres, c'est avant le javascript que c'est généré donc tu peux le faire ici ! 

In [2]:
root = Path(".").resolve()
data_folder = root / "data"
file_path = data_folder / "wine_links.csv"

In [ ]:
session = create_session()

Voir `MLFlow` pour un package qui permet d'obtenir rapidement des metrics de Machine Learning

Donc on importe plutôt les données du folder data en polars puis on retransforme en liste

In [123]:
all_wine_urls = list(
    chain.from_iterable(pl.read_csv(file_path, has_header=False).to_numpy().tolist())
)

In [124]:
len(all_wine_urls)

4341

In [125]:
all_wine_urls = all_wine_urls[0:500]

In [126]:
len(all_wine_urls)

500

Enfin, on extrait tout le contenu des pages avec `extract_all_pages`. Ensuite..c'est à toi ! 🤪

In [ ]:
all_html_pages = extract_all_pages(session, all_wine_urls)

In [129]:
len(all_html_pages)

499

In [4]:
from src.modules.mystical_soup import *

In [131]:
create_json(all_html_pages)

Export en JSON réalisé avec succès dans D:\Cours Mecen (M2)\Machine Learning\Wine Scraping\data !


**IDEE** : Faire une carte de la provenance géographique des vins

- slicer par type de vin
- par prix
- etc.

In [3]:
from src.modules.bear_cleaner import *

In [4]:
df = pl.read_json(data_folder / "vins.json")

Pour tout ce qui est texte *gouts, oeil, nez, bouche, etc.* $\Rightarrow$ Utiliser le package `NTLK` (qui fait du Natural Language Processing)

In [5]:
df = super_pipe(df)

In [21]:
vin_select = "PINOT NOIR 2019 LAS PIZARRAS - ERRAZURIZ"

In [24]:
df.select("name", "unit_price").filter(pl.col("name") == vin_select).select(
    "unit_price"
).item()

94.9

In [8]:
import plotly.express as px

In [52]:
grouped_df = (
    df.group_by("country", "iso_code")
    .count()
    .sort("count", descending=True)
    .filter(pl.col("country") != "12,5 % vol")
)

In [54]:
grouped_df

country,iso_code,count
str,str,u32
"""France""","""FRA""",3203
"""Italie""","""ITA""",226
"""Espagne""","""ESP""",166
"""Argentine""","""ARG""",59
"""Etats-Unis""","""USA""",58
"""Australie""","""AUS""",51
"""Chili""","""CHL""",49
"""Allemagne""",null,44
"""Afrique du Sud…",null,29


In [53]:
px.bar(grouped_df, x="country", y="count")

utiliser `df.write_database()` pour écrire le polars en base de données